Get list of gameIds

In [88]:
import pandas as pd
def get_ids(season,game_type):
    #get url with proper gameId, convert to json
    season_input = str(season)+str(season+1)
    url = f"https://statsapi.web.nhl.com/api/v1/schedule"
    response = requests.get(url, params={"season" : season_input,"Content-Type": "application/json","gameType" : game_type})
    schedule_data =  response.json()
    
    #dates = filter(lambda a : (a["result"]["event"] == "Goal"),shotData["liveData"]["plays"]["allPlays"]) 
    dates = schedule_data['dates']
    game_ids = []
    for date in dates:
        for game in date['games']:
            game_ids.append(game['gamePk'])
    return game_ids
x = get_ids(2019,'P')

In [87]:
#game type inputs:
def game_types():
    response = requests.get('https://statsapi.web.nhl.com/api/v1/gameTypes')
    game_types =  response.json()
    types = []
    for code in game_types:
        types.append(f"{code['description']} : {code['id']}")
    return types
game_types()
                 

['Pre-season : PR',
 'Regular season : R',
 'Playoffs : P',
 'All-Star game : A',
 'All-Star Women game : WA',
 'Olympic game : O',
 'World cup of hockey exhibition/preseason games  : WCOH_EXH',
 'World cup of hockey preliminary games  : WCOH_PRELIM',
 'World cup of hockey semifinals and finals  : WCOH_FINAL']

get goal data from a game

In [216]:
def get_shot_data(shot):
    shot_data = {
        **shot['coordinates'],
        'event' : shot['result']['event'],
        'secondaryType' : shot['result'].get('secondaryType'),
        'shooter' : shot['players'][0]['player']['fullName'] if shot['result']['event'] != "Blocked Shot" else shot['players'][1]['player']['fullName'],
        'blocker' : shot['players'][0]['player']['fullName'] if shot['result']['event'] == "Blocked Shot" else 'None',
        'team' : shot['team']['name'], #note: refers to team of blocker for blocked shots
        'strength' : shot['result']['strength']['name'] if shot['result']['event'] == 'Goal' else 'None',
        'period' : shot['about']['period'],
        'game_winning_goal' : shot['result'].get('gameWinningGoal'),
        'empty_net' : shot['result'].get('emptyNet'),
    }
    #empty net goals mean there may not be a goalie in net
    goalie_info = list(filter(lambda a : a['playerType'] == 'Goalie',shot['players']))
    shot_data['goalie'] = goalie_info[0]['player']['fullName'] if len(goalie_info) > 0 else 'None'

    #Assists
    for i,player in enumerate(list(filter(lambda a : a['playerType'] == 'Assist',shot['players']))):
        shot_data[f"assist_{i+1}"] = player['player']['fullName']
    return shot_data

Read all the data from the api into a list

In [225]:
import requests
import pandas as pd

#function to retrieve all goal locations from a team in a given season
def season_shots(season,game_type):
    shot_data = []
    for game_id in get_ids(season,game_type):
        #get url with proper gameId, convert to json
        API_URL = f"https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live"
        response = requests.get(API_URL, params={"Content-Type": "application/json"})
        game_data = response.json()
        shots = filter(lambda a : a["result"]["event"] == "Goal" 
               or a["result"]["event"] == "Shot" 
               or a["result"]["event"] == "Missed Shot"
               or a["result"]["event"] == "Blocked Shot" ,game_data["liveData"]["plays"]["allPlays"]) 
        
        for shot in shots:
            shot_data.append({**get_shot_data(shot),'game_id' : game_id})
    return shot_data

In [227]:
%%time
x = season_shots(2022,'R')

Wall time: 3min 36s


Export

In [553]:
df = pd.DataFrame(x)
df.to_csv('season_shots.csv')